# Will Gasser | 2/20/2025 | MNIST NN

This is a simple NN with back propogation to formalize my understanding of ML and NN concepts.

I am limiting myself to having imports: numpy, random, time (just for stats), pandas (just for data read), and matplotlib for visualization after.

Let us see how fast I can complete this, oh and no LLMs for any functions, I will note if I use it for syntax.

Init commit now: 2/20/2025 @ 9:14 EST

Finish: TBD

In [2]:
import numpy as np
import random
import time
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
TRAIN_DATA = np.array(pd.read_csv('mnist_train.csv'))
TEST_DATA = np.array(pd.read_csv('mnist_test.csv'))

In [ ]:
# DATA FUNCTIONS

DATA_SIZE = np.shape(TRAIN_DATA)
BATCH_INDICES = list(range(0, DATA_SIZE[0]))

def shuffle_batch() -> None:
    random.shuffle(BATCH_INDICES)
    
def get_one_example(index: int) -> np.ndarray:
    return TRAIN_DATA[index]

def get_label(data: np.ndarray) -> int:
    return data[0]
    
def get_features(data: np.ndarray) -> np.ndarray:
    return data[np.newaxis, 1:] / 255.0



In [ ]:
# FORWARD PASS / BACKWARD PROPOGATION FUNCTIONS

def activation_sigmoid(Z: np.ndarray) -> np.ndarray: # Z is activations
    return (1 / (1+ np.exp(-Z)))

def forward_pass(activation_in: np.ndarray, layers: list, bias: list) -> np.ndarray:
        activation_out = []
        for i in range(len(layers)): # at each neural layer
            layer = layers[i] # specific layer, specific bias
            B = bias[i]
            activaiton_in = activation_sigmoid(np.dot(activation_in, layer) + B)
            activation_out.append(activation_in)
            
        for output, i in enumerate(activation_in.flatten()):
            print(f'{output}: {i}')
        
        return activation_out
                
def back_propogation(activation_vectors: list, layers: list, bias: list) -> None:
            

            

In [ ]:
def training(data_size: int, epochs: int, batch_size: int, data: np.ndarray, layers: list, bias: list) -> None:
    if (data_size % batch_size != 0):
        raise ValueError(f'ERROR: Data of size {data_size} is not divisible by Batch Size of {batch_size}')
    
    BATCHES = data_size / batch_size
    
    print(f'Commencing Training with:\nEpochs: {epochs}\nBatches: {BATCHES}\nBatch Size: {batch_size}')
    
    total_time = time.time
    for epoch in range(epochs):
        epoch_time = time.time
        activation_vectors = []
        
        for batch in range(BATCHES):
            for batch_index in range(batch_size):
                activation_vectors.append(forward_pass(data[batch_index]))
            back_propogation(activation_vectors, layers, bias)
            
        print(f'EPOCH {epoch} complete in {epoch_time - time.time}(s)')
    print(f'TRAINING COMPLETE in time: {total_time - time.time}(s)')
        

In [47]:
'''
###################################################################
### THIS CELL SETS THE MODEL WEIGHTS/BIAS BACK TO RANDOM VALUES ###
###################################################################
'''

# MODEL INSTANTIATION

weights784 = np.random.uniform(low=-1.0, high=1.0, size=(784, 256)) # INPUT LAYER
bias256 = np.zeros((1,256))

weights256 = np.random.uniform(low=-1.0, high=1.0, size=(256, 128)) # Hidden layer 1
bias128 = np.zeros((1,128))

weights128 = np.random.uniform(low=-1.0, high=1.0, size=(128, 64)) # Hidden layer 2
bias64 = np.zeros((1,64))

weights64= np.random.uniform(low=-1.0, high=1.0, size=(64, 10)) # OUTPUT
bias10 = np.zeros((1,10))

layers = [weights784, weights256, weights128, weights64]
bias = [bias256, bias128, bias64, bias10]

In [ ]:
# SET OUR BATCH SIZES AND EPOCHS
BATCH_SIZE = np.size(TRAIN_DATA) / 10
EPOCHS = 1

In [ ]:
# DATA PRE-PROCESSING
print(f'Data processing, shuffling batch of size {len(BATCH_INDICES)}.')
start = time.time()
shuffle_batch()
shuffle_time = time.time() - start
print(f'Data shuffling complete, took {shuffle_time} seconds.')

test_data = get_one_example(BATCH_INDICES[0])
test_example = get_features(test_data)
test_label = get_label(test_data)



# TEST FORWARD PROP
print('COMMENCING FORWARD PROPOGATION TEST')

forward_pass(test_example, layers, bias)

print(f'Label: {test_label}')
print('COMPLETE')

Data processing, shuffling batch of size 59999.
Data shuffling complete, took 0.02061748504638672 seconds.
COMMENCING FORWARD PROPOGATION TEST
(1, 10)
0: 0.030215178225351327
1: 0.4277305515516519
2: 0.8774139792803056
3: 0.228274923814295
4: 0.8708545646972903
5: 0.231017873418005
6: 0.23784773674567408
7: 0.09263769349503741
8: 0.00946463769519365
9: 0.8059917865761259
Label: 5
COMPLETE
